In [1]:
import pandas as pd
import sklearn as sk
import nltk 
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer

### Importación de los datos y limpieza

In [2]:
import json
# Cargar la configuración desde un archivo JSON
with open("configura_local.json", encoding="utf-8") as f:
    config = json.load(f)

print("Ruta cargada:", config["data_path"])

Ruta cargada: C:/Users/JOSE DANIEL M/OneDrive/Documentos/Universidad/Aprendizaje Estadístico/Aprendizaje Estadístico/RottenTomatoes/rotten_tomatoes_critic_reviews.csv


In [3]:
file_path = json.load(open("configura_local.json", encoding="utf-8"))["data_path"]
df = pd.read_csv(file_path)

In [4]:
df.head(5)

,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
0,m/0814255,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...
1,m/0814255,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff..."
2,m/0814255,NaN,False,FILMINK (Australia),Fresh,NaN,2010-02-09,With a top-notch cast and dazzling special eff...
3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...
4,m/0814255,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,What's really lacking in The Lightning Thief i...


In [5]:
#Filtrado de columnas que se van a utilizar
columnas = ['review_score','review_content']
df = df[columnas]
df.head(5)

,review_score,review_content
0,NaN,A fantasy adventure that fuses Greek mythology...
1,NaN,"Uma Thurman as Medusa, the gorgon with a coiff..."
2,NaN,With a top-notch cast and dazzling special eff...
3,3.5/5,Whether audiences will get behind The Lightnin...
4,NaN,What's really lacking in The Lightning Thief i...


In [6]:
# Eliminación de filas con valores nulos
df.dropna(inplace=True)

In [7]:
# Diccionario de conversión de letras a escala de 5 como fracción
letter_to_score = {
    'A': '5/5',
    'A-': '4.7/5',
    'B+': '4.3/5',
    'B': '4/5',
    'B-': '3.7/5',
    'C+': '3.3/5',
    'C': '3/5',
    'C-': '2.7/5',
    'D+': '2.3/5',
    'D': '2/5',
    'D-': '1.7/5',
    'F': '1/5',
}

# Función para reemplazar calificaciones de letra
def convert_score(score):
    score_str = str(score).strip()
    if score_str in letter_to_score:
        return letter_to_score[score_str]
    return score_str  # deja igual si ya es una fracción como "3.5/5"

# Aplica la función a la columna de calificaciones
df['review_score'] = df['review_score'].apply(convert_score)

In [8]:
df.head(5)

,review_score,review_content
3,3.5/5,Whether audiences will get behind The Lightnin...
6,1/4,Harry Potter knockoffs don't come more transpa...
7,3.5/5,"Percy Jackson isn't a great movie, but it's a ..."
8,4/5,"Fun, brisk and imaginative"
9,3/5,"Crammed with dragons, set-destroying fights an..."


In [8]:
# Convertir fracciones a número decimal
def fraction_to_float(score_str):
    try:
        num, den = score_str.split('/')
        return float(num) / float(den)
    except:
        return None  # para manejar errores o valores inesperados

df['review_score'] = df['review_score'].apply(fraction_to_float)


In [10]:
df.head(5)

,review_score,review_content
3,0.70,Whether audiences will get behind The Lightnin...
6,0.25,Harry Potter knockoffs don't come more transpa...
7,0.70,"Percy Jackson isn't a great movie, but it's a ..."
8,0.80,"Fun, brisk and imaginative"
9,0.60,"Crammed with dragons, set-destroying fights an..."


In [9]:
# Calcular la mediana
median_score = df['review_score'].median()
print("Mediana de review_score:", round(median_score, 3))

Mediana de review_score: 0.66


In [10]:
# Clasificar como 'positivo' si el score es mayor que la mediana, si no, 'negativo'
df['sentiment'] = df['review_score'].apply(
    lambda x: 'positivo' if x >= median_score else 'negativo'
)

In [13]:
df

,review_score,review_content,sentiment
3,0.70,Whether audiences will get behind The Lightnin...,positivo
6,0.25,Harry Potter knockoffs don't come more transpa...,negativo
7,0.70,"Percy Jackson isn't a great movie, but it's a ...",positivo
8,0.80,"Fun, brisk and imaginative",positivo
9,0.60,"Crammed with dragons, set-destroying fights an...",negativo
...,...,...,...
1130006,0.80,As a spectacular war film with a powerful mora...,positivo
1130013,0.70,"Seen today, it's not only a startling indictme...",positivo
1130014,0.86,A rousing visual spectacle that's a prequel of...,positivo
1130015,0.70,"A simple two-act story: Prelude to war, and th...",positivo


In [11]:
# Conteo de valores positivos y negativos de la columna 'sentiment'
print(df['sentiment'].value_counts())

sentiment
positivo    386805
negativo    371904
Name: count, dtype: int64


No queda balanceado exacto, pero resulta suficiente

In [12]:
# Filtrar las columnas que se van a utilizar
df = df[['review_content', 'sentiment']]

# Cambio de etiquetas de sentimiento a valores numéricos 
# 'positivo' a 1 y 'negativo' a 0
target_map = {'positivo': 1, 'negativo': 0}
df['sentiment'] = df['sentiment'].map(target_map)
df.head(5)

,review_content,sentiment
3,Whether audiences will get behind The Lightnin...,1
6,Harry Potter knockoffs don't come more transpa...,0
7,"Percy Jackson isn't a great movie, but it's a ...",1
8,"Fun, brisk and imaginative",1
9,"Crammed with dragons, set-destroying fights an...",0


Vamos entonces a crear nuestra división Train/Test

In [21]:
# Definir X e y
X = df['review_content']
y = df['sentiment']

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #80% entrenamiento, 20% testeo

# Verificación
print("Entrenamiento:", X_train.shape, y_train.shape)
print("Testeo:", X_test.shape, y_test.shape)


Entrenamiento: (606967,) (606967,)
Testeo: (151742,) (151742,)


### Modelamiento

## Modelo Meme
Para empezar, vampos a  immplementar un modelo muy sencillo que tiene 50% de error. Nuestro trabajo será en las siguientes implementaciones mejorar los errores paulatinamente.

In [22]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

# Genera etiquetas aleatorias (0 o 1) para el conjunto de prueba
y_pred_random = np.random.choice([0, 1], size=len(y_test))

# Evalúa el desempeño del modelo aleatorio
print("Accuracy:", accuracy_score(y_test, y_pred_random))
print("F1 Score:", f1_score(y_test, y_pred_random))

Accuracy: 0.5009687495881167
F1 Score: 0.5070693920062492


Como era de esperarse el modelo solo calsifica bien el 50% de los datos

## Modelo de Vocabularios
Haremos un conteo de frecuencias, empíricamente y apoyándonos de la teoría encontraremos las expresiones que más se repiten en cada clase. Ejemplo: "awful" siendo muy presente en los textos negativos y "great" en textos positivos

Pero primero debemos hacer otra capa de preprocesamiento de los datos. Por el momoento, las observaciones en la base de datos df consisten solo de texto. Le haremos una tokenización, dividirlo en palabras, frases o caracteres, para hacer luego el conteo de frecuencias. Nos centraremos en eliminar puntuaciones indeseadas (mantenemos "!" y "?") y categorías gramaticales como artículos, preposiciones, conjunciones,..., también conectores que no alteren la polaridad a

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Descargar recursos si no están
#nltk.download('punkt')
#.download('stopwords')

In [28]:
#import nltk
#nltk.download('punkt_tab')
def tokenizar_con_signos(texto):
    # Separar ? y ! con espacio si están pegados a una palabra
    texto = re.sub(r"([!?])", r" \1 ", texto)
    texto = re.sub(r"\s+", " ", texto)  # limpiar espacios duplicados
    return word_tokenize(texto.lower())
tokenizar_con_signos("That was great!")  # → ['that', 'was', 'great', '!']
tokenizar_con_signos("Not good?")        # → ['not', 'good', '?']


['not', 'good', '?']

In [33]:
# Función para limpiar texto y tokenizar
def limpiar_texto(texto):
    """
    Preprocesa un texto: lo pasa a minúsculas, elimina puntuación (excepto ? y !),
    tokeniza, y opcionalmente elimina stopwords.
    """
    # 1. Minúsculas
    texto = texto.lower()

    # 2. Eliminar puntuación excepto ? y !
    texto = re.sub(r"([!?])", r" \1 ", texto) #Separa ? y ! con espacio
    texto = re.sub(r"\s+", " ", texto)  # limpiar espacios duplicados
    texto = re.sub(r"[^a-zA-Z\s!?]", "", texto) # elimina todo excepto letras, números, espacios, ? y !

    # 3. Tokenizar
    tokens = word_tokenize(texto)

    # 4. Eliminar stopwords:
    sw = set(stopwords.words('english')) - {"not", "no", "nor", "don", "won",  "but"}  # conservamos palabras clave
    tokens = [word for word in tokens if word not in sw]

    return tokens

print(limpiar_texto("Wow! I did **not** expect that... 10/10 movie? Absolutely great! But my friend didn't like it."))

['wow', '!', 'not', 'expect', 'movie', '?', 'absolutely', 'great', '!', 'but', 'friend', 'didnt', 'like']


In [ ]:
from collections import Counter

# Tokenizar con la función que creamos personalizada limpiar_texto
X_train['tokens'] = X_train.apply(limpiar_texto)

In [41]:
# Aplanamos la lista de listas
todos_los_tokens_positivos = [
    token for tokens, label in zip(X_train["tokens"], y_train) if label == 1 for token in tokens
]

todos_los_tokens_negativos = [
    token for tokens, label in zip(X_train["tokens"], y_train) if label == 0 for token in tokens
]

# Contamos frecuencias
conteo_unigramas_pos = Counter(todos_los_tokens_positivos)
conteo_unigramas_neg = Counter(todos_los_tokens_negativos)

In [52]:
# Mostramos los 20 más comunes
print("Positivos:")
conteo_unigramas_pos.most_common(10)
#print("Negativos:")
#conteo_unigramas_neg.most_common(10)

Positivos:


[('but', 51786),
 ('film', 50362),
 ('movie', 32725),
 ('one', 28735),
 ('not', 25700),
 ('like', 18357),
 ('story', 18082),
 ('best', 14571),
 ('good', 12462),
 ('films', 12329)]